In [26]:
import sys
from pathlib import Path

project_path = Path().resolve().parent
sys.path.append(str(project_path))

In [27]:
import pandas as pd
import plotly.express as px

from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose

from fetch_data import create_denfensivos_agricolas_df

In [28]:
COLUMNS_TO_REMOVE = [
    "valor_fob_usd",
    "valor_frete_usd",
    "valor_frete_usd",
    "valor_cif_usd",
    "qtd_estatistica",
    "valor_seguro_usd",
]

## 1. **Pre-processing**

In [29]:
df_pest = create_denfensivos_agricolas_df()

/home/hcordeiro/personal/venv/dviz/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'api-comexstat.mdic.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings

/home/hcordeiro/personal/venv/dviz/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'api-comexstat.mdic.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings



In [30]:
df_pest_explore = df_pest.drop(columns=COLUMNS_TO_REMOVE).copy()
df_pest_explore.head()

,ano,mes,pais,estado,via_de_transporte,unidade_receita_federal,id_ncm,descritor_ncm,unidade_medida,peso_liq_kg,class
0,2024,10,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,Outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,18907628,OTHERS
1,2024,10,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089324,"Herbicida à base de glifosato ou seus sais, de...",QUILOGRAMA LIQUIDO,22475354,OTHERS
2,2024,06,Estados Unidos,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089199,"Outros inseticidas, apresentados de outro modo",QUILOGRAMA LIQUIDO,642837,OTHERS
3,2024,07,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,Outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,12113051,OTHERS
4,2024,08,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,Outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,15503590,OTHERS


In [31]:
df_pest_explore["descritor_ncm"].nunique(), df_pest_explore["id_ncm"].nunique()

(98, 113)

### Lower description strings

In [32]:
df_pest_explore["descritor_ncm"] = df_pest_explore["descritor_ncm"].str.lower()

### Enforce dtypes

In [33]:
df_pest_explore["peso_liq_kg"] = df_pest_explore["peso_liq_kg"].astype(float)

In [34]:
df_pest_explore.dtypes

ano                         object
mes                         object
pais                        object
estado                      object
via_de_transporte           object
unidade_receita_federal     object
id_ncm                      object
descritor_ncm               object
unidade_medida              object
peso_liq_kg                float64
class                       object
dtype: object

### Add dt column

In [35]:
df_pest_explore["dt"] = pd.to_datetime(df_pest_explore['ano'] + '-' + df_pest_explore['mes'] + '-01')


In [36]:
df_pest_explore.loc[:, "extracted_at"] = datetime.today()


In [37]:
date_cols_to_drop = ["ano", "mes"]

In [38]:
df_pest_explore = df_pest_explore.drop(columns=date_cols_to_drop)
df_pest_explore.head()

,pais,estado,via_de_transporte,unidade_receita_federal,id_ncm,descritor_ncm,unidade_medida,peso_liq_kg,class,dt,extracted_at
0,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,18907628.0,OTHERS,2024-10-01,2025-01-26 20:13:08.574113
1,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089324,"herbicida à base de glifosato ou seus sais, de...",QUILOGRAMA LIQUIDO,22475354.0,OTHERS,2024-10-01,2025-01-26 20:13:08.574113
2,Estados Unidos,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089199,"outros inseticidas, apresentados de outro modo",QUILOGRAMA LIQUIDO,642837.0,OTHERS,2024-06-01,2025-01-26 20:13:08.574113
3,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,12113051.0,OTHERS,2024-07-01,2025-01-26 20:13:08.574113
4,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,15503590.0,OTHERS,2024-08-01,2025-01-26 20:13:08.574113


### Check duplicates and nans

In [39]:
df_pest_explore.isna().any()

pais                       False
estado                     False
via_de_transporte          False
unidade_receita_federal    False
id_ncm                     False
descritor_ncm              False
unidade_medida             False
peso_liq_kg                False
class                      False
dt                         False
extracted_at               False
dtype: bool

In [40]:
df_pest_explore.duplicated().any()

np.False_

### Create class map

In [41]:
# one hot categories
df_pest_explore["is_domissanitario"] = False
df_pest_explore["is_herbicide"] = False
df_pest_explore["is_inseticide"] = False
df_pest_explore["is_fungicide"] = False
df_pest_explore["is_ddt"] = False
df_pest_explore["is_ddt"] = False

domissanit_cond = df_pest_explore["descritor_ncm"].str.contains("domissanit")
df_pest_explore.loc[domissanit_cond, "is_domissanitario"] = True

herbicide_cond = df_pest_explore["descritor_ncm"].str.contains("herbicida|germina")
df_pest_explore.loc[herbicide_cond, "is_herbicide"] = True

inseticide_cond = df_pest_explore["descritor_ncm"].str.contains("inseticid")
df_pest_explore.loc[inseticide_cond, "is_inseticide"] = True

fungicide_cond = df_pest_explore["descritor_ncm"].str.contains("fungicid")
df_pest_explore.loc[fungicide_cond, "is_fungicide"] = True

ddt_cond = df_pest_explore["descritor_ncm"].str.contains("ddt")
df_pest_explore.loc[ddt_cond, "is_ddt"] = True

In [42]:
df_pest_explore["is_multiple_categories"] = False

pesticide_classes_cols = ["is_herbicide", "is_inseticide", "is_fungicide"]
multiple_cond = df_pest_explore[pesticide_classes_cols].sum(axis=1) > 1
df_pest_explore.loc[multiple_cond, "is_multiple_categories"] = True

In [43]:
df_pest_explore.head()

,pais,estado,via_de_transporte,unidade_receita_federal,id_ncm,descritor_ncm,unidade_medida,peso_liq_kg,class,dt,extracted_at,is_domissanitario,is_herbicide,is_inseticide,is_fungicide,is_ddt,is_multiple_categories
0,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,18907628.0,OTHERS,2024-10-01,2025-01-26 20:13:08.574113,False,True,False,False,False,False
1,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089324,"herbicida à base de glifosato ou seus sais, de...",QUILOGRAMA LIQUIDO,22475354.0,OTHERS,2024-10-01,2025-01-26 20:13:08.574113,False,True,False,False,False,False
2,Estados Unidos,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089199,"outros inseticidas, apresentados de outro modo",QUILOGRAMA LIQUIDO,642837.0,OTHERS,2024-06-01,2025-01-26 20:13:08.574113,False,False,True,False,False,False
3,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,12113051.0,OTHERS,2024-07-01,2025-01-26 20:13:08.574113,False,True,False,False,False,False
4,China,São Paulo,MARITIMA,0817800 - PORTO DE SANTOS,38089329,outros herbicidas apresentados de outro modo,QUILOGRAMA LIQUIDO,15503590.0,OTHERS,2024-08-01,2025-01-26 20:13:08.574113,False,True,False,False,False,False


In [44]:
df_pest_explore.to_csv("./2025_01_06_processed_export_data.csv")

## 2. **Resampling, Aggregating**

In [55]:
value_keys = ["peso_liq_kg"] 
# resample_key = "dt"
# valid_grouper_keys = ["pais", "estado", "via_de_transporte", "id_ncm", "class"]
grouper_keys = ["dt", "class"]

_df_keys = value_keys + grouper_keys

In [56]:
grouped_df = df_pest_explore[_df_keys].groupby(grouper_keys).sum().reset_index()
grouped_df

,dt,class,peso_liq_kg
0,1997-01-01,DESINFETANTES,62556.0
1,1997-01-01,FUNGICIDES,220946.0
2,1997-01-01,HERBICIDES,620160.0
3,1997-01-01,OTHERS,23288.0
4,1997-01-01,PESTICIDES,593550.0
...,...,...,...
1015,2024-10-01,UNCLEAR,7636.0
1016,2024-11-01,OTHERS,67866801.0
1017,2024-11-01,UNCLEAR,16200.0
1018,2024-12-01,OTHERS,65970256.0


In [47]:
fig = px.bar(
    grouped_df,
    x="dt",
    y="peso_liq_kg",
    color="class"
)

fig.show()

In [ ]:
value_keys = ["peso_liq_kg"]
dt_keys = ["dt"]
one_hot_keys = ['is_domissanitario', 'is_herbicide', 'is_inseticide', 'is_fungicide', 'is_ddt']

_df_keys = value_keys + dt_keys + one_hot_keys

# Aggregate the sum of 'value' for each class
class_sums = df_pest_explore[_df_keys].groupby(dt_keys + one_hot_keys).sum().reset_index()

In [58]:
# Melt the DataFrame to have one row per class
melted_agg_df = class_sums.melt(id_vars=dt_keys + value_keys, 
                         value_vars=one_hot_keys, 
                         var_name='new_class', 
                         value_name='is_present')

melted_agg_df = melted_agg_df[melted_agg_df['is_present'] == 1]  # keep only values of classes that existed in the og data

melted_agg_df

,dt,peso_liq_kg,new_class,is_present
4,1997-01-01,169926.0,is_domissanitario,True
5,1997-01-01,234778.0,is_domissanitario,True
10,1997-02-01,18070.0,is_domissanitario,True
11,1997-02-01,135922.0,is_domissanitario,True
12,1997-02-01,80742.0,is_domissanitario,True
...,...,...,...,...
9778,2004-07-01,0.0,is_ddt,True
9953,2006-12-01,0.0,is_ddt,True
10271,2010-06-01,0.0,is_ddt,True
10381,2011-08-01,0.0,is_ddt,True


In [59]:
fig = px.bar(
    melted_agg_df,
    x="dt",
    y="peso_liq_kg",
    color="new_class"
)

fig.show()

In [48]:
monthly_ts_df = df_pest_explore[_df_keys].resample("ME", on=resample_key).sum()
monthly_ts_df

In [49]:
decomposed_ts = seasonal_decompose(monthly_ts_df)

In [50]:
decomposed_ts.plot()